# Проект "Предобратка данных"

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Открываем файл, сохраняем его в переменной `data` выводим первые 10 строк на экран.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Посмотри общую информацию о данных.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


После применения метода info() видим, что датасет имеет 21525 строк и 12 столбцов (с различными типами данных: float64, int64, object). 
Названия столбцов с информацией об их содержании:

* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита.

Обратим внимание, что в столбцах days_employed и total_income есть пропуски. Т.к. пропусков одинаковое количество, возможно, есть взаимосвязь между ними.

**Выводы**

На данном этапе мы видим несколько проблем:
1. Пропуски в столбцах `days_employed` и `total_income`
2. Дубликаты в столбце `education` с различным регистром
3. В столбце `purpose` дубликаты по смыслу (например, "на проведение свадьбы" и "сыграть свадьбу")

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим названия столбцов на наличие пробелов и прочих символов, затрудняющих доступ к данным.

In [3]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Видим, что все они имеют корректные названия.

Проверим количество пропусков в столбце `days_employed`

In [4]:
data['days_employed'].isnull().sum()

2174

Убедимся, что в столбце `total_income` такое же количество пропущенных значений.

In [5]:
data['total_income'].isnull().sum()

2174

Количество строк с пропущенными значениями в столбцах "Общий трудовой стаж" и "Ежемесячный доход" - по 2174.

Заполним столбец `days_employed` медианными значениями с учётом типа занятости. Для начала избавимся от отрицательных значений.

In [6]:
data['days_employed'] = data['days_employed'].abs()
display(data['days_employed'])

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

Напишем функцию, которая принимает на вход тип занятости, а возвращает среднее значение трудового стажа в днях.

In [7]:
def median_days_employed(income_type):
    return data.loc[data['income_type'] == income_type, 'days_employed'].median()

Передадим функции типы занятости с пропущенными знаениями.

In [8]:
data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'сотрудник'), 'days_employed'] = median_days_employed('сотрудник')

In [9]:
data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'пенсионер'), 'days_employed'] = median_days_employed('пенсионер')

In [10]:
data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'компаньон'), 'days_employed'] = median_days_employed('компаньон')

In [11]:
data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'госслужащий'), 'days_employed'] = median_days_employed('госслужащий')

In [12]:
data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'предприниматель'), 'days_employed'] = median_days_employed('предприниматель')

Проверим, исчезли ли пропуски из столбца `days_employed`.

In [13]:
data.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Теперь заменим пропуски на медианные значения с учётом типа занятости в столбце `total_income`.

In [14]:
def median_total_income(income_type):
    return data.loc[data['income_type'] == income_type, 'total_income'].median()

In [15]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = median_total_income('сотрудник')

In [16]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = median_total_income('пенсионер')

In [17]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = median_total_income('компаньон')

In [18]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = median_total_income('госслужащий')

In [19]:
data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = median_total_income('предприниматель')

In [20]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Замена прошла успешно, в столбцах больше нет пропущенных значений. Данные могли быть пропущены ошибочно, либо на момент заполнения таблицы не было информации о трудовом стаже и заработке клиента.

### Замена типа данных

Заменим вещественный тип данных на целочисленный с помощью метода `astype()`. Выбираем данный метод, т.к. все пропуски в столбцах `days_employed` и `total_income` мы заменили на числовые значения, и ошибок быть не должно.

In [22]:
data['days_employed'] = data['days_employed'].astype('int')

In [23]:
data['total_income'] = data['total_income'].astype('int')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Замена прошла успешно, dtypes: float64 заменились на int64. 

### Обработка дубликатов

На этапе ознакомления с данными мы заметили, что в столбце `education` есть дубликаты. Рассмотрим этот столбец подробнее.

In [25]:
data['education'].value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Видим, что присутствуют дубликаты с учётом регистра. На самом деле категорий всего 5: начальное, среднее, неоконченное высшее, высшее и ученая степень.
Приведем их все к единообразному виду.

In [26]:
data['education'] = data['education'].str.lower()

Проверим результат.

In [27]:
data['education'].value_counts() 

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

 Теперь посмотрим, есть ли в нашей таблице полные дубликаты.

In [28]:
data.duplicated().sum()

71

Мы обнаружили 71 позицию. Мы можем посмотреть на уникальные строки, которые имеют дубликаты.

In [29]:
data[data.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,365213,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,365213,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,365213,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,1574,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,1574,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


Мы можем предположить, что произошли задвоения, т.к. очень маловероятно нахождение в таблице идентичных клиентов, учитывая вариативность показателей в столбцах `days_employed` и `total_income`. Поэтому удалим дубли.

In [30]:
data = data.drop_duplicates().reset_index(drop=True)

Проверим ещё раз наличие дублей.

In [31]:
data.duplicated().sum()

0

Удаление дублей прошло успешно.

### Вывод

Нам удалось избавиться от дубликатов с помощью метода string.lower(). Мы увидели проблему с регистром, поэтому привели все категории к нижнему регистру. Такая проблема могла возникнуть, если данные заполнялись разными людьми: каждый сотрудник заносил данные о клиентах, как ему было удобнее.

### Лемматизация

Проведём лемматизацию в столбце `purpose`. Для начала импортируем библиотеку pymystem3.

In [32]:
from pymystem3 import Mystem
m = Mystem()

Создадим датафрейм со столбцами `purpose` и `lemmas`.

In [33]:
purposes = pd.DataFrame(data = data['purpose'], columns = ['purpose', 'lemmas'])
purposes = purposes.drop_duplicates().reset_index(drop=True)

In [34]:
def lemmatization(text):
    lemmas = ' '.join(m.lemmatize(text))
    return lemmas

In [35]:
purposes['lemmas'] = purposes['purpose'].apply(lemmatization)
purposes

,purpose,lemmas
0,покупка жилья,покупка жилье \n
1,приобретение автомобиля,приобретение автомобиль \n
2,дополнительное образование,дополнительный образование \n
3,сыграть свадьбу,сыграть свадьба \n
4,операции с жильем,операция с жилье \n
5,образование,образование \n
6,на проведение свадьбы,на проведение свадьба \n
7,покупка жилья для семьи,покупка жилье для семья \n
8,покупка недвижимости,покупка недвижимость \n
9,покупка коммерческой недвижимости,покупка коммерческий недвижимость \n


Напишем функцию, которая на вход будет принимать строковое значение, внутри делать лемматизацию этой строки и получать список лемм, а возвращать конкретное значение категории.

In [36]:
def purpose_category(row):
    lemmas = row['lemmas']
    if 'автомобиль' in lemmas:
        return 'покупка автомобиля'
    elif ('недвижимость' in lemmas) or ('жилье' in lemmas):
        return 'покупка недвижимости'
    elif 'свадьба' in lemmas:
        return 'свадьба' 
    elif 'образование' in lemmas:
        return 'образование'
    return 'другая категория'

Применим эту функцию с помощью метода `apply` к столбцу с целями кредита, а результат запишем в новый столбец по которому планируется в дальнейшем группировка.

In [37]:
purposes['purpose_category'] = purposes.apply(purpose_category, axis = 1)

С помощью метода `merge` объеденим оба датафрейма.

In [38]:
data = data.merge(purposes, on='purpose', how='left')

In [39]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба


### Вывод

Мы провели лемматизацию, теперь каждый вариант целей кредита будет соотносится с определённой категорией: "свадьба", "покупка недвижимости", "образование" или "покупка автомобиля".

### Категоризация данных

Словари нужны, чтобы оптимально хранить информацию о категориях. Каждой категории соответствует определенный номер. В дальнейшем его можно использовать вместо текстового названия категории в таблице.
Однин из словарей в нашем датафрейме - пара столбцов `family_status` и `family_status_id`.

In [40]:
data.pivot_table(index = 'family_status', values = 'family_status_id').to_dict()['family_status_id']

{'Не женат / не замужем': 4,
 'в разводе': 3,
 'вдовец / вдова': 2,
 'гражданский брак': 1,
 'женат / замужем': 0}

Мы видим здесь "ключи" ('в разводе', 'Не женат / не замужем' и т.д.) и "значения" (от 0 до 4). То есть каждой категории присвоено своё числовое обозначение.
Ещё один словарь - пара столбцов `education` и `education_id`.

In [41]:
data.pivot_table(index = 'education', values = 'education_id').to_dict()['education_id']

{'высшее': 0,
 'начальное': 3,
 'неоконченное высшее': 2,
 'среднее': 1,
 'ученая степень': 4}

Также мы можем добавить категории для столбца `children`. Сначала посотрим, какие данные есть в этом столбце.

In [42]:
data['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обращаем внимание, что очень много клиентов имеют 20 детей, а также есть некорректные значения -1. 
Скорее всего здесь ошибка, предположительно там, где детей "20", должно быть число "2". А там где "-1" соответсвенно просто "1".
Заменим неправильные значения в столбце.

In [43]:
data['children'] = data['children'].replace({-1 : 1, 20 : 2})

In [44]:
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Теперь напишем функцию. Разделим клиентов на 2 группы в зависимости от наличия детей.

In [45]:
def having_children(children):
    if children < 1:
        return 'нет детей'
    return 'есть дети'

С помощью метода `apply()` создадим новый столбец в нашем датафрейме.

In [46]:
data['having_children'] = data['children'].apply(having_children)

In [47]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_category,having_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,покупка недвижимости,есть дети
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,покупка автомобиля,есть дети
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,покупка недвижимости,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n,покупка недвижимости,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n,покупка недвижимости,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n,свадьба,есть дети
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n,покупка недвижимости,нет детей


Новый столбец появился в нашем датафрейме, посмотрим какое количество детей в каждой из категорий.

In [48]:
data['having_children'].value_counts()

нет детей    14091
есть дети     7363
Name: having_children, dtype: int64

### Вывод

Мы провели категоризацию данных для столбца `children` и выделили словари в нашем датафрейме.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Посмотрим сколько в целом людей имеют задолженности по кредитам.

In [49]:
data['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Построим сводную таблицу, в которой отразим столбцы `having_children` и `debt`

In [50]:
children_pvt = data.groupby('having_children').agg({'debt': ['count', 'sum']})
children_pvt

debt      
                 count   sum
having_children             
есть дети         7363   678
нет детей        14091  1063

Из таблицы видим, что у 678 из 7376 людей у кого есть дети присутствует и задолженность.
В то же время у 1063 из 14149 людей есть задолженность и нет детей.
Для наглядности посмотрим сколько это будет в процентах.

In [51]:
children_pvt['percent'] = children_pvt['debt']['sum'] / children_pvt['debt']['count'] * 100
children_pvt

debt         percent
                 count   sum          
having_children                       
есть дети         7363   678  9.208203
нет детей        14091  1063  7.543822

### Вывод

Примерно 9% людей имеют задолженность среди тех, у кого есть дети и около 7% среди тех, у кого детей нет. 
В целом нельзя сказать, что есть зависимость между наличием детей и возвратом кредита в срок.
Но можно предположить, что людям, у которых нет детей немного проще возвращать кредит вовремя, чем людям с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Построим сводную таблицу, чтобы посмотреть на количество задолженностей у людей с разлчиным семейным положением.

In [52]:
family_status_pvt = data.groupby('family_status').agg({'debt': ['count', 'sum']})
family_status_pvt

debt     
                       count  sum
family_status                    
Не женат / не замужем   2810  274
в разводе               1195   85
вдовец / вдова           959   63
гражданский брак        4151  388
женат / замужем        12339  931

Добавим столбец с процентным соотношением.

In [53]:
family_status_pvt['percent'] = family_status_pvt['debt']['sum'] / family_status_pvt['debt']['count'] * 100
family_status_pvt

debt        percent
                       count  sum          
family_status                              
Не женат / не замужем   2810  274  9.750890
в разводе               1195   85  7.112971
вдовец / вдова           959   63  6.569343
гражданский брак        4151  388  9.347145
женат / замужем        12339  931  7.545182

### Вывод

Наиболее высокий показатель задолженности мы видим на позициях "не женат/не замужем" и "гражданский брак". Возможно, это в основном более молодые клиенты. Соответственно уровень дохода у них может быть меньше. Также, возможно, в эту группу входят студенты. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Прежде чем мы сможем ответить на этот вопрос нам нужно столбец `total_income` разбить на категории. Для начала посмотрим на минимальный и максимальный уровень дохода в таблице.

In [54]:
data['total_income'].min()

20667

In [55]:
data['total_income'].max()

2265604

Теперь разделим значения в этом столбце на несколько категорий: "До 100 тыс.", "До 150 тыс.", "До 200 тыс." и "Свыше 200 тыс.".

In [56]:
def income_group(total_income):
    if total_income < 100001:
        return 'До 100 тыс.'
    if total_income < 150001:
        return 'До 150 тыс.'
    if total_income < 200001:
        return 'До 200 тыс.'
    return 'Свыше 200 тыс.'

In [57]:
data['income_group'] = data['total_income'].apply(income_group)

In [58]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,purpose_category,having_children,income_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n,покупка недвижимости,есть дети,Свыше 200 тыс.
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n,покупка автомобиля,есть дети,До 150 тыс.
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n,покупка недвижимости,нет детей,До 150 тыс.
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n,образование,есть дети,Свыше 200 тыс.
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n,свадьба,нет детей,До 200 тыс.


Посмотрим на количество значений в каждой из групп.

In [59]:
data['income_group'].value_counts()

До 150 тыс.       7160
Свыше 200 тыс.    5067
До 200 тыс.       4764
До 100 тыс.       4463
Name: income_group, dtype: int64

Построим сводную таблицу и вычислим процентное соотношение.

In [60]:
income_group_pvt = data.groupby('income_group').agg({'debt': ['count', 'sum']})
income_group_pvt

debt     
               count  sum
income_group             
До 100 тыс.     4463  354
До 150 тыс.     7160  624
До 200 тыс.     4764  405
Свыше 200 тыс.  5067  358

In [61]:
income_group_pvt['percent'] = income_group_pvt['debt']['sum'] / income_group_pvt['debt']['count'] * 100
income_group_pvt

debt        percent
               count  sum          
income_group                       
До 100 тыс.     4463  354  7.931884
До 150 тыс.     7160  624  8.715084
До 200 тыс.     4764  405  8.501259
Свыше 200 тыс.  5067  358  7.065325

### Вывод

Заметной разницы в соотношении должников по категориям нет. Можно отметить, что в целом их количество меньше в группе с доходом свыше 200 тыс. руб.

- Как разные цели кредита влияют на его возврат в срок?

Построим сводную таблицу, чтобы посмотреть на количество задолженностей c учётом целей кредита.

In [62]:
purpose_category_pvt = data.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum'])
purpose_category_pvt

,count,sum
,debt,debt
purpose_category,,
образование,4013,370
покупка автомобиля,4306,403
покупка недвижимости,10811,782
свадьба,2324,186


In [63]:
purpose_category_pvt['percent'] = purpose_category_pvt['sum']['debt'] / purpose_category_pvt['count']['debt'] * 100
purpose_category_pvt

,count,sum,percent
,debt,debt,
purpose_category,,,
образование,4013,370,9.220035
покупка автомобиля,4306,403,9.359034
покупка недвижимости,10811,782,7.233373
свадьба,2324,186,8.003442


### Вывод

Можно сделать вывод, что люди, берущие кредит на образование или покупку автомобиля, в целом чаще оказываются в списке должников.
Возможно, это связано с тем, что более молодые люди берут кредиты на такие нужды. У них пока ещё нет стабильной работы и карьерных достижений, поэтому купить автомобиль, не прибегая к кредиту, сложнее. И, коенчно, более остро стоит вопрос образования  с целью получения более прибыльной работы.

### Шаг 4. Общий вывод

По итогам исследования можно сделать следующие выводы:

1. Клиенты с достатком свыше 200 тыс.руб. чаще погашают кредиты в срок.
2. Кредиты, взятые на проведение свадьбы или покупку жилья скорее погасятся в срок, нежели кредиты на автомобиль или образование.
3. Если клиент не женат/не замужем или состоит в гражданском браке, вероятность того, что кредит не будет погашен в срок возрастает.
4. В то же время, если у клиента есть дети, то вероятность того, что он отдаст кредит вовремя понижается. Возможно, это связано с наличием дополнительных расходов на семью.
